Manually creating stim log for estim trains

In [1]:
import io
import sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import copy
import time

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib notebook

## Load data

In [5]:
# rec_folder = r'F:\testing\mouse000000\old_test_2023-09-07_14-07-36\experiment1\recording1' # old log style
rec_folder = r'W:\mouse735049\aw_stim_train_2024-05-24_10-36-53\experiment1\recording1'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical stimulation


In [6]:
sync_data = exp._load_sync_dataset()

# Estim log

In [7]:
esweep_rising = sync_data.get_edges(keys=('estim_sweep',), kind='rising', units='seconds')
esweep_falling = sync_data.get_edges(keys=('estim_sweep',), kind='falling', units='seconds')
esync_rising = sync_data.get_edges(keys=('estim_sync',), kind='rising', units='seconds')
esync_falling = sync_data.get_edges(keys=('estim_sync',), kind='falling', units='seconds')

In [8]:
total_trials = len(esync_rising)
print('Total e-stim trials: %d' % total_trials)
num_sweeps = len(esweep_rising)
print('Number of e-stim sweeps: %d' % num_sweeps)
trials_per_sweep = int(total_trials / num_sweeps)
print('Trials per sweep: %d' % trials_per_sweep)

Total e-stim trials: 150
Number of e-stim sweeps: 2
Trials per sweep: 75


#### Read multichannel systems stim file

In [9]:
print(exp.stim_instruction_files['electrical'])

['W:\\mouse735049\\aw_stim_train_2024-05-24_10-36-53\\experiment1\\recording1\\stimulation_protocol_1pulse_50tr_25uA_6.5-7.5_ISI.txt', 'W:\\mouse735049\\aw_stim_train_2024-05-24_10-36-53\\experiment1\\recording1\\stimulation_protocol_2pulse_50tr_25uA_6.5-7.5_ISI.txt']


In [10]:
mcs_dfs = []
for efile in exp.stim_instruction_files['electrical']:
    with open(efile) as file:
        estim_txt = file.read() # open estim file and read in text
    estim = estim_txt.split('channel:') # split and read txt file
    d = {}
    for item in estim:
        if 'value' in item:
            key = 'channel'+str(item[1])
            d[key]=item.split('\n')
    channel1 = d['channel1'][1:]
    mcs_dfs.append(pd.read_csv(io.StringIO('\n'.join(channel1)), delim_whitespace=True))
    
ch1_df = pd.concat(mcs_dfs, axis=0, sort=False)
print(len(ch1_df))

550


In [11]:
list_stim_type = []
list_stim_amp = []
list_stim_dur = []
for i in range(1, len(ch1_df)):
    if ch1_df.value.iloc[i-1] < 0 and ch1_df.value.iloc[i] > 0:
        list_stim_type.append('biphasic')
        list_stim_amp.append(ch1_df.value.iloc[i])
        list_stim_dur.append(ch1_df.time.iloc[i-1] + ch1_df.time.iloc[i])
estim_log = pd.DataFrame({
    'stim_type': list_stim_type,
    'parameter': list_stim_amp,
    'duration': np.array(list_stim_dur) * 1e-6
})

In [12]:
print(len(estim_log))
estim_log.head()

150


,stim_type,parameter,duration
0,biphasic,25,0.0004
1,biphasic,25,0.0004
2,biphasic,25,0.0004
3,biphasic,25,0.0004
4,biphasic,25,0.0004


In [13]:
if len(estim_log) == trials_per_sweep:
    print('estim_log matches trials per sweep')
    estim_log = pd.concat([estim_log] * num_sweeps, axis='index', ignore_index=True)
elif len(estim_log) == len(esync_rising):
    print('estim_log matches total trials')
    estim_log = estim_log
else:
    print('estim_log does not match total trials or trials per sweep')

estim_log matches total trials


In [14]:
estim_log['onset'] = esync_rising
estim_log['offset'] = esync_falling

### Assign single/double pulse trials

In [15]:
stimISI = np.insert(np.diff(estim_log['onset']), 0, 10.0)
print(len(stimISI))
print(stimISI[:10])

150
[10.       7.00191  7.40154  7.36679  7.23956  7.39854  7.36697  6.87858
  6.82846  7.69953]


In [16]:
evind = 0
event_ID = 0
event_sep = 1. # temporal separation of discrete events (s)

evtypelist = []
evIDlist = []
while evind < len(estim_log):
    tempevent = [stimISI[evind]]
    evIDlist.append(event_ID)
    evind += 1
    while (evind < len(estim_log)) and (stimISI[evind] < event_sep):
        tempevent.append(stimISI[evind])
        evIDlist.append(event_ID)
        evind += 1
    evtypelist.extend([str(len(tempevent)) + 'pulse'] * len(tempevent))
    event_ID += 1

In [17]:
print(len(evtypelist))
print(len(evIDlist))

150
150


In [18]:
estim_log['event_type'] = evtypelist
estim_log['event_ID'] = evIDlist

In [19]:
estim_log.tail()

,stim_type,parameter,duration,onset,offset,event_type,event_ID
145,biphasic,25,0.0004,1360.86847,1360.86887,2pulse,97
146,biphasic,25,0.0004,1368.07969,1368.08009,2pulse,98
147,biphasic,25,0.0004,1368.28009,1368.28049,2pulse,98
148,biphasic,25,0.0004,1375.79816,1375.79856,2pulse,99
149,biphasic,25,0.0004,1375.99856,1375.99896,2pulse,99


In [20]:
estim_log['sweep'] = np.zeros(len(estim_log), dtype=int) - 1
for i in range(len(esweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    estim_log.loc[(estim_log['onset'] >= esweep_rising[i]-5) & (estim_log['onset'] <= esweep_falling[i]+2), 'sweep'] = i

In [21]:
estim_log.head()

,stim_type,parameter,duration,onset,offset,event_type,event_ID,sweep
0,biphasic,25,0.0004,339.97544,339.97584,1pulse,0,0
1,biphasic,25,0.0004,346.97735,346.97775,1pulse,1,0
2,biphasic,25,0.0004,354.37889,354.37929,1pulse,2,0
3,biphasic,25,0.0004,361.74568,361.74608,1pulse,3,0
4,biphasic,25,0.0004,368.98524,368.98564,1pulse,4,0


In [22]:
all_stim_log = pd.concat([estim_log], axis=0, sort=False).sort_values(by='onset', axis=0, ignore_index=True)
all_stim_log = all_stim_log[['event_type', 'event_ID', 'stim_type', 'parameter', 'onset', 'offset', 'duration', 'sweep']]

In [23]:
all_stim_log.tail()

,event_type,event_ID,stim_type,parameter,onset,offset,duration,sweep
145,2pulse,97,biphasic,25,1360.86847,1360.86887,0.0004,1
146,2pulse,98,biphasic,25,1368.07969,1368.08009,0.0004,1
147,2pulse,98,biphasic,25,1368.28009,1368.28049,0.0004,1
148,2pulse,99,biphasic,25,1375.79816,1375.79856,0.0004,1
149,2pulse,99,biphasic,25,1375.99856,1375.99896,0.0004,1


In [24]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

# Sensory stim

#### Set parameters and load sync file

#### Load experiment log file

#### Add photodiode times as onsets/offsets (if available)

#### Add duration (for old log) and sweep number

#### Check it

# Put them all together and save